In [2]:
from urllib.parse import urlencode, unquote, quote_plus
from datetime import datetime,timedelta
from urllib.request import urlopen
import pandas as pd
import xmltodict # 결과가 xml 형식으로 반환된다. 이것을 dict 로 바꿔주는 라이브러리다
import requests
import pickle
import urllib
import folium
import json
import os

# Corona19 Data

In [3]:
# 어제 날짜와 오늘날짜를 구하기 위해서  datetime과 timedelta를 사용
yester = datetime.today() - timedelta(1)
yseter =  yester.strftime("%Y%m%d")
now_today = datetime.today().strftime("%Y%m%d")

my_api_key = 'Wcgao4rabgjIy%2FySdMTfP%2BO9dVXqUYCAUyMenwN%2BWFsj%2BtXvR3bIrUhW%2B7lUfnn3UYxDJ4uSlIes1UmY4URRdQ%3D%3D'

# 서비스 url 주소
url = 'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19SidoInfStateJson'

# 서비스에 필요한 파라미터 모음
queryParams = '?' + \
'ServiceKey=' + '{}'.format(my_api_key) + \
'&pageNo='+ '1' + \
'&numOfRows='+ '999' + \
'&startCreateDt={}&endCreateDt={}'.format(yseter,now_today)

final_url = url + queryParams

res = urllib.request.urlopen(final_url)
json_str = res.read().decode("utf-8")


jsonString = json.dumps(xmltodict.parse(json_str), indent = 4, ensure_ascii=False)
jsonString = jsonString.replace('createDt', '등록일시분초').replace('deathCnt', '사망자수').replace('defCnt', '확진자수').replace('gubun', '시도명(한글)').replace('gubunCn', '시도명(중국어)').replace('gubunEn', '시도명(영어)').replace('incDec', '전일대비증가수').replace('isolClearCnt', '격리해제수').replace('isolIngCnt', '격리중환자수').replace('localOccCnt', '지역발생수').replace('overFlowCnt', '해외유입수').replace('qurRate', '10만명당발생률').replace('seq', '게시글번호').replace('stdDay', '기준일시').replace('updateDt', '수정일시분초')

json_object = json.loads(jsonString)


res2 = pd.DataFrame(json_object["response"]["body"]["items"]["item"])
# display(res2)

pickle.dump(res2, open('corona19_data.pickle', 'wb'))

corona19_data = pickle.load(open('corona19_data.pickle', 'rb'))

display(corona19_data.head(5))

등록일시분초 사망자수  확진자수 시도명(한글) 시도명(한글)Cn         시도명(한글)En  \
0  2021-02-19 09:36:09.611    3  2912      검역       隔離區         Lazaretto   
1  2021-02-19 09:36:09.611    1   556      제주        济州              Jeju   
2  2021-02-19 09:36:09.611   10  2138      경남        庆南  Gyeongsangnam-do   
3  2021-02-19 09:36:09.611   70  3164      경북        庆北  Gyeongsangbuk-do   
4   2021-02-19 09:36:09.61    8   810      전남        全南      Jeollanam-do   

  전일대비증가수 격리해제수 격리중환자수 지역발생수 해외유입수 10만명당발생률 게시글번호               기준일시 수정일시분초  
0      10  2230    679     0    10        -  7721  2021년 02월 19일 00시   null  
1       3   535     20     3     0    82.89  7720  2021년 02월 19일 00시   null  
2       9  2038     90     9     0    63.61  7719  2021년 02월 19일 00시   null  
3      22  2974    120    22     0   118.84  7718  2021년 02월 19일 00시   null  
4      10   743     59    10     0    43.44  7717  2021년 02월 19일 00시   null

In [4]:
now_today2 = datetime.today().strftime("%Y-%m-%d")



# "corona19_data['등록일시분초']"의 원래 dtype는 object 이다.
# 문자열(object)을 datetime64 타입으로 변경하면 요일부터 다양한 추가 정보를 이용할 수 있습니다.
corona19_data['등록일시분초'] = pd.to_datetime(corona19_data['등록일시분초'], format='%Y-%m-%d %H:%M:%S', errors='raise')
corona19_data['등록일시분초'] = corona19_data['등록일시분초'].dt.date.astype(str)

corona19_data2 = corona19_data[corona19_data['등록일시분초'] == now_today2]

corona19_data3 = corona19_data2.loc[:, ['등록일시분초', '시도명(한글)', '전일대비증가수']]

corona19_data4 = corona19_data3[~(corona19_data3['시도명(한글)'] == '검역') & ~(corona19_data3['시도명(한글)'] =='합계')] 
print(corona19_data4)


        등록일시분초 시도명(한글) 전일대비증가수
1   2021-02-19      제주       3
2   2021-02-19      경남       9
3   2021-02-19      경북      22
4   2021-02-19      전남      10
5   2021-02-19      전북       3
6   2021-02-19      충남      24
7   2021-02-19      충북      18
8   2021-02-19      강원       7
9   2021-02-19      경기     189
10  2021-02-19      세종       1
11  2021-02-19      울산      12
12  2021-02-19      대전       4
13  2021-02-19      광주       4
14  2021-02-19      인천      38
15  2021-02-19      대구      11
16  2021-02-19      부산      16
17  2021-02-19      서울     180


# Preprocessing administrative districts CODE&NAME

In [5]:
admin_districts = pd.read_excel('AdminDistricts.xlsx', sheet_name='1. 총괄표(현행)')
display(admin_districts.head(10))

print(type(admin_districts))


Unnamed: 0 행정구역분류 총괄표(2019.10.1.기준) Unnamed: 2 Unnamed: 3 Unnamed: 4  \
0         NaN                      NaN        NaN        NaN        NaN   
1         NaN                      대분류         시도        중분류        시군구   
2         NaN                       11      서울특별시        NaN        NaN   
3         NaN                       11      서울특별시      11010        종로구   
4         NaN                       11      서울특별시      11010        종로구   
5         NaN                       11      서울특별시      11010        종로구   
6         NaN                       11      서울특별시      11010        종로구   
7         NaN                       11      서울특별시      11010        종로구   
8         NaN                       11      서울특별시      11010        종로구   
9         NaN                       11      서울특별시      11010        종로구   

  Unnamed: 5 Unnamed: 6        Unnamed: 7 Unnamed: 8  
0        NaN        NaN               NaN        NaN  
1        소분류        읍면동             영문 표기      한자 표기  
2        NaN        NaN             Seoul      서울特別市  
3        NaN        NaN         Jongno-gu        鍾路區  
4    1101053        사직동        Sajik-dong        社稷洞  
5    1101054        삼청동    Samcheong-dong        三淸洞  
6    1101055        부암동         Buam-dong        付岩洞  
7    1101056        평창동  Pyeongchang-dong        平倉洞  
8    1101057        무악동         Muak-dong        毋岳洞  
9    1101058        교남동       Gyonam-dong        橋南洞

<class 'pandas.core.frame.DataFrame'>


In [6]:
ad_dis2 = admin_districts.loc[1:, '행정구역분류 총괄표(2019.10.1.기준)':'Unnamed: 2']
display(ad_dis2.head(5))


행정구역분류 총괄표(2019.10.1.기준) Unnamed: 2
1                      대분류         시도
2                       11      서울특별시
3                       11      서울특별시
4                       11      서울특별시
5                       11      서울특별시

In [7]:
ad_dis3 = ad_dis2.rename(columns=ad_dis2.iloc[0])
ad_dis4 = ad_dis3.drop(ad_dis3.index[0])
display(ad_dis4.head(5))

대분류     시도
2  11  서울특별시
3  11  서울특별시
4  11  서울특별시
5  11  서울특별시
6  11  서울특별시

In [8]:
ad_dis5 = ad_dis4.drop_duplicates(['대분류'])
display(ad_dis5)

대분류       시도
2      11    서울특별시
452   NaN      NaN
453    21    부산광역시
677    22    대구광역시
826    23    인천광역시
991    24    광주광역시
1093   25    대전광역시
1179   26    울산광역시
1242   29  세종특별자치시
1264   31      경기도
1856   32      강원도
2069   33     충청북도
2239   34     충청남도
2465   35     전라북도
2726   36     전라남도
3047   37     경상북도
3406   38     경상남도
3739   39  제주특별자치도

In [9]:
# 결측치 제거하기. axis=0은 행, axis=1은 열
ad_dis6 = ad_dis5.dropna(axis=0)
display(ad_dis6)
# ad_dis6.to_csv('ad_dis.csv', mode='w')


대분류       시도
2     11    서울특별시
453   21    부산광역시
677   22    대구광역시
826   23    인천광역시
991   24    광주광역시
1093  25    대전광역시
1179  26    울산광역시
1242  29  세종특별자치시
1264  31      경기도
1856  32      강원도
2069  33     충청북도
2239  34     충청남도
2465  35     전라북도
2726  36     전라남도
3047  37     경상북도
3406  38     경상남도
3739  39  제주특별자치도

In [10]:
ad_dis_reborn = pd.read_csv('ad_dis.csv')

def name_changer(name):
    if len(name) >= 5:
        name = name[:2]
    elif len(name) == 4:
        name = name[0] + name[2]
    else:
        name = name[:-1]
    return name 

# aa.apply(lambda x: x['시도'])

ad_dis_reborn['시도'] = ad_dis_reborn['시도'].apply(lambda x : name_changer(x))

ad_dis_reborn

Unnamed: 0  대분류  시도
0            2   11  서울
1          453   21  부산
2          677   22  대구
3          826   23  인천
4          991   24  광주
5         1093   25  대전
6         1179   26  울산
7         1242   29  세종
8         1264   31  경기
9         1856   32  강원
10        2069   33  충북
11        2239   34  충남
12        2465   35  전북
13        2726   36  전남
14        3047   37  경북
15        3406   38  경남
16        3739   39  제주

In [39]:
merge_data = pd.merge(corona19_data4, ad_dis_reborn, left_on='시도명(한글)', right_on='시도')
final_data = merge_data[['대분류', '전일대비증가수']]

final_data['전일대비증가수'] = final_data['전일대비증가수'].astype(float)
final_data2 = final_data.sort_values(by='전일대비증가수', axis=0, ascending=False)
final_data2.columns = ['Code', 'Population']
final_data2['Code'] = final_data2['Code'].astype(str)
print(final_data2)
print(type(final_data2['Code'][0]))


   Code  Population
8    31       189.0
16   11       180.0
13   23        38.0
5    34        24.0
2    37        22.0
6    33        18.0
15   21        16.0
10   26        12.0
14   22        11.0
3    36        10.0
1    38         9.0
7    32         7.0
11   25         4.0
12   24         4.0
4    35         3.0
0    39         3.0
9    29         1.0
<class 'str'>


In [49]:
state_geo = 'TL_SCCO_CTPRVN_WGS84_renewal.json'



m = folium.Map(location=[36, 127], tiles="OpenStreetMap", zoom_start=7)

m.choropleth(
 geo_data=state_geo,
 name='choropleth',
 data=final_data2,
 columns=('Code', 'Population'),
 key_on='feature.properties.CTPRVN_CD',
 fill_color='YlGn',
 fill_opacity=0.8,
 line_opacity=0.9,
 legend_name='Population Rate (%)'
)

m




In [23]:
geoJson_for_dump = json.load(open('TL_SCCO_CTPRVN_WGS84.json', 'r'), encoding='cp949')

for i in range(17):
    geoJson_for_dump["features"][i]["properties"]["CTPRVN_CD"] = admin_districts_changer(geoJson_for_dump["features"][i]["properties"]["CTPRVN_CD"])


with open('TL_SCCO_CTPRVN_WGS84_renewal.json', 'w') as fi:
    json.dump(geoJson_for_dump, fi)


# for i in range(17):
#     num = aa["features"][i]["properties"]["CTPRVN_CD"]
#     name = aa["features"][i]["properties"]["CTP_ENG_NM"]
#     print(f'num: {num}, name: {name}')

# aaa = json.dumps(aa, indent=2)
# print(aaa)


In [20]:
def admin_districts_changer(num):
    if num == '42':
        num = '32'
    elif num == '41':
        num = '31'
    elif num == '48':
        num = '38'
    elif num == '47':
        num = '37'
    elif num == '29':
        num = '24'
    elif num == '27':
        num = '22'
    elif num == '30':
        num = '25'
    elif num == '26':
        num = '21'
    elif num == '36':
        num = '29'
    elif num == '31':
        num = '26'
    elif num == '28':
        num = '23'
    elif num == '46':
        num = '36'
    elif num == '45':
        num = '35'
    elif num == '50':
        num = '39'
    elif num == '44':
        num = '34'
    elif num == '43':
        num = '33'
    
    return num


In [ ]:
{
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "geometry": {
        "type": "MultiPolygon",
        "coordinates": [
          [
            [
              [
                127.15,
                38.307
              ],
              [
           126.423,
                37.063
              ]
            ]
          ]
        ]
      },
      "properties": {
        "CTPRVN_CD": "44",
        "CTP_ENG_NM": "Chungcheongnam-do",
        "CTP_KOR_NM": "\ucda9\uccad\ub0a8\ub3c4"
      }